<table>
    <tr>
        <td><img src="./img/Macc.png" width="auto"/></td>
        <td>
            <table><tr>
            <h1 style="color:blue;text-align:center">Lógica para Ciencias de la Computación</h1></td>
            </tr></table>   
        <td>&nbsp;</td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Taller</p></tp>
            <tp><p style="font-size:150%;text-align:center">Solución de problemas</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---

# Objetivo <a class="anchor" id="inicio"></a>

En clase hemos visto el siguiente esquema del procedimiento para solucionar problemas:

<img src="./img/mapa.png" width="400px"/>

Implementaremos este esquema en Python usando las herramientas desarrolladas hasta este momento.

# Secciones

1. [Problema ilustrativo.](#prob)
2. [Implementación de las restricciones.](#forms)
3. [Visualización.](#vis)
4. [Solución mediante SATtabla.](#sat)
5. [Un problema más difícil.](#prob2)
6. [Situaciones más complejas y comparación de tiempos.](#comp)

# Problema de ejemplo <a class="anchor" id="prob"></a>

([Volver al inicio](#inicio))

Comenzaremos con el problema presentado en las diapositivas de clase, que repetimos aquí por conveniencia:

<img src="./img/problema.png" width="400px"/>

Las restricciones y su representación usando lógica proposicional son las siguientes:

1. Debe haber exactamente tres caballos en el tablero:

$$\bigvee_{(x_1,y_1)\in\texttt{Casillas}}\bigvee_{(x_2,y_2)\neq(x_1,y_1)}\bigvee_{{(x_3,y_3)\neq(x_1,y_1)\\(x_3,y_3)\neq(x_2,y_2)}}\left(CenC_{(x_1,y_1)}\wedge CenC_{(x_2,y_2)}\wedge CenC_{(x_3,y_3)}\right)\wedge\bigwedge_{(u,v)\neq(x_1,y_1)\\(u,v)\neq(x_2,y_2)\\(u,v)\neq(x_3,y_3)}\neg CenC_{(u,v)}\Bigr)$$

2. Ningún caballo debe poder atacar a otro.

\begin{align*}
&CenC_{(0,0)}\to \neg(CenC_{(1,2)} \vee CenC_{(2,1)})\\
&CenC_{(1,0)}\to \neg(CenC_{(0,2)} \vee CenC_{(2,2)})\\
&CenC_{(2,0)}\to \neg(CenC_{(1,2)} \vee CenC_{(0,1)})\\
&CenC_{(0,1)}\to \neg(CenC_{(2,2)} \vee CenC_{(2,0)})\\
&CenC_{(2,1)}\to \neg(CenC_{(0,2)} \vee CenC_{(0,0)})\\
&CenC_{(0,2)}\to \neg(CenC_{(1,0)} \vee CenC_{(2,1)})\\
&CenC_{(1,2)}\to \neg(CenC_{(0,0)} \vee CenC_{(2,0)})\\
&CenC_{(2,2)}\to \neg(CenC_{(0,1)} \vee CenC_{(1,0)})\\
\end{align*}


3. Debe haber un caballo en la casilla $(1,2)$.

$$CenC_{(1,2)}$$

## Implementación de las restricciones <a class="anchor" id="forms"></a>

([Volver al inicio](#inicio))

En la librería `Problemas` hemos implementado las tres reglas. Puede correr el siguiente código para visualizar el resultado, y puede entrar a la librería para explorar la implementación:

In [ ]:
from Problemas import Caballos
from Logica import *

In [ ]:
c = Caballos()

Observe que el objeto tiene un atributo `reglas`, en el cual está una lista con las tres reglas implementadas en Python:

In [ ]:
c.reglas

La clase `Caballos` crea de una vez el descriptor, mediante el cual podemos decodificar las letras:

In [ ]:
c.CenC.escribir('ć')

También podemos revisar regla por regla. Observe que las fórmulas incluyen un método `ver` para visualizar más fácilmente las reglas:

In [ ]:
A = inorder_to_tree(c.reglas[1])
print(A.ver(c.CenC))

In [ ]:
A = inorder_to_tree(c.reglas[0])
print(A.ver(c.CenC))

## Visualización <a class="anchor" id="vis"></a>

([Volver al inicio](#inicio))

Una parte muy importante de la clase `Caballos` es el método que permite visualizar el tablero, dada una interpretación de las reglas proposicionales. Esto es lo que nos permite entender la solución del problema.

In [ ]:
I = {c.CenC.P([0,2]):True, c.CenC.P([1,2]):True, c.CenC.P([2,2]):True}
c.visualizar(I)

## Solución mediante SATtabla <a class="anchor" id="sat"></a>

([Volver al inicio](#inicio))

Ahora podemos ver el corazón de la solución a este y todos los demas problemas que desarrollemos: un SATsolver. Usaremos el SATtabla que implementamos anteriormente.

In [ ]:
%%time
A = inorder_to_tree(Ytoria(c.reglas))
I = A.SATtabla()

Podemos decodificar el diccionario `I` obtenido para ver la solución:

In [ ]:
if I != None:
    for k in I:
        print(c.CenC.escribir(k), I[k])
else:
    print('¡No hay solución!')

O, mejor aún, podemos visualizar la solución:

In [ ]:
if I != None:
    c.visualizar(I)
else:
    print('¡No hay solución!')

Vemos que la solución encontrada por SATtabla es muy rápida.

---

## Un problema más difícil <a class="anchor" id="prob2"></a>

([Volver al inicio](#inicio))

El propósito de esta sección es tratar de resolver un problema mediante el procedimiento que hemos descrito anteriormente. No obstante, el código presentará algunos problemas que tendremos que afrontar. Esto nos permitirá entrenarnos para afrontar las dificultades que se presentarán en la elaboración del proyecto.

Retomaremos aquí el problema con el que estudiamos la representación de situaciones. Buscamos llenar todas las casillas en una tabla 2x2 con un número de 0 a 3, sin repetir. Por ejemplo:

![ejemplo](img/tabla.png)

Tenemos tres restricciones que debemos implementar para resolver el problema:

1. Un número sólo está en una casilla.

$$\bigwedge_{x\in\{0,1\}}\bigwedge_{y\in\{0,1\}}\bigwedge_{n\in Numeros}\left(OenCasilla_{x,y,n}\to\neg\left(\bigvee_{(u,v)\neq (x,y)} OenCasilla_{u,v,n}\right)\right)$$

2. No hay más de un número en una casilla.

$$\bigwedge_{x\in\{0,1\}}\bigwedge_{y\in\{0,1\}}\bigwedge_{n\in Numeros}\left(OenCasilla_{x,y,n}\to\neg\left(\bigvee_{m\neq n} OenCasilla_{x,y,m}\right)\right)$$

3. Debe haber por lo menos un número en una casilla.

$$\bigwedge_{x\in\{0,1\}}\bigwedge_{y\in\{0,1\}}\left(\bigvee_{n\in Numeros}OenCasilla_{x,y,n}\right)$$


### Errores sintácticos

La implementación, con algunos errores, está en la librería `Problemas`.

In [ ]:
from Problemas import Rejilla
from Logica import *

Instanciamos el objeto:

In [ ]:
r = Rejilla(N=2, M=2)

Y ahora intentamos visualizar la regla 1:

In [ ]:
print(r.reglas[0])
A = inorder_to_tree(r.reglas[0])
print(A.ver(r.NenC))

Este error aparece en la línea que usa el método `ver` de las fórmulas. No obstante, el método `ver` es correcto. Es decir, el problema no está aquí. Esto lo sabemos porque el error nos dice que un objeto de tipo `None` no tiene el atributo `ver`. Así pues, el error es que `A` es `None`. Corrámos solo la función `inorder_to_tree` para comprobarlo:

In [ ]:
A = inorder_to_tree(r.reglas[0])
print(A)

Este error en `inorder_to_tree` surge porque la cadena no está bien escrita. Usualmente se debe a que hay un paréntesis faltante o mal puesto.

**Ejercicio 1:**

Revise el siguiénte código que genera la regla 1 y corrija el error. Observe que a la generación de fórmulas le hace falta incluir un paréntesis.

In [ ]:
def regla1(self):
    casillas_num = [(n,x,y) for n in range(self.N*self.M) for x in range(self.N) for y in range(self.M)]
    lista = []
    for c in casillas_num:
        n,x,y = c
        otras_casillas = [(x1,y1) for x1 in range(self.N) for y1 in range(self.M) if (x1,y1) != (x,y)]
        lista_o = []
        for k in otras_casillas:
            lista_o.append(self.NenC.P([n,*k]))
        form = '(' + self.NenC.P([*c]) + '>-' + Otoria(lista_o)
        lista.append(form)
    return Ytoria(lista)

setattr(Rejilla, "regla1", regla1)

---

### Errores semánticos

Un tipo de error más complicado de ver es cuando la regla está correctamente escrita, pero no representa la restricción de manera correcta.

Por ejemplo, la regla 3 está escrita correctamente:

In [ ]:
r = Rejilla(N=2)
A = inorder_to_tree(r.reglas[2])
print(A.ver(r.NenC))

No obstante, ella no cumple el papel que debe cumplir. Esto se puede detectar al tratar de resolver el problema con esta regla y obtener resultados indeseados.

In [ ]:
%%time
r = Rejilla(N=2)
A = inorder_to_tree(Ytoria(r.reglas))
I = A.SATtabla()
if I != None:
    r.visualizar(I)
else:
    print('¡No hay solución!')

Si corremos la regla 3 solamente, obtenemos lo siguiente:

In [ ]:
%%time
r = Rejilla(N=2)
A = inorder_to_tree(r.reglas[2])
I = A.SATtabla()
if I != None:
    r.visualizar(I)
else:
    print('¡No hay solución!')

**Ejercicio 2:**

Corrija la regla 3 para que represente la restricción de manera apropiada.

In [ ]:
def regla3(self):
    casillas = [(x,y) for x in range(self.N) for y in range(self.M)]
    lista = []
    for c in casillas:
        lista_o = []
        for n in range(self.N*self.M):
            lista_o.append(self.NenC.P([n,*c]))
        lista.append(Ytoria(lista_o))
    return Ytoria(lista)

setattr(Rejilla, "regla3", regla3)

In [ ]:
%%time
r = Rejilla(N=2)
A = inorder_to_tree(Ytoria(r.reglas))
I = A.SATtabla()
if I != None:
    r.visualizar(I)
else:
    print('¡No hay solución!')

---

# Situaciones más complejas y comparación de tiempos <a class="anchor" id="comp"></a>

([Volver al inicio](#inicio))

Observe que podemos cambiar los parámetros `N` y `M` para cambiar el tamaño de la rejilla:

In [ ]:
r = Rejilla(N=3, M=2)
I = {r.NenC.P([1,0,0]):True, r.NenC.P([1,1,0]):True}
r.visualizar(I)

Tal vez queramos aplicar el mismo algoritmo de solución a este problema más grande. No obstante, el tiempo de ejecución es prohibitivo:

In [ ]:
%%time
r = Rejilla(N=3, M=2)
A = inorder_to_tree(Ytoria(r.reglas))
I = A.SATtabla()
r.visualizar(I)

Este es un problema muy serio. Ni siquiera el algoritmo de los tableros semánticos puede venir a rescatarnos (observe que el objeto `Formula` ya trae implementado el `SATtableaux`):

In [ ]:
%%time
r = Rejilla(N=3, M=2)
A = inorder_to_tree(Ytoria(r.reglas))
I = A.SATtableaux()
r.visualizar(I)

---

## En este notebook usted aprendió

1. El esquema de solución de problemas mediante lógica proposicional.
2. Cada problema requiere una definición particular de sus restricciones y su implementación en lógica proposicional.
3. Cualquiera de estos problemas se resuelve (por lo menos de manera teórica) mediante el mismo algoritmo: un SATsolver.
4. Los SATsolvers que tenemos hasta este momento no son para nada eficientes, así que necesitamos algoritmos más poderosos.